## Problem statement

Next, take one of the samples from training dataset and use PINN [3] for estimating the value of
G. Report the CPU/GPU time taken. You may use equation 4 for computing the residual loss.

In [9]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

### Device and seed

In [12]:
# device = tf.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(f"Device used: {device}")
np.random.seed(seed=1234)
tf.random.set_seed(1234)

tf.config.list_physical_devices('GPU')

[]

### Load Data

In [13]:
## Load dataset
relative_csv_path = "./../src/data/raw/Dataset.csv"
split_ratio = 0.2

# Load different temperatures samples 
fields = ['T1', 'T2', 'T3', 'T4' ,'T5', 'T6' ,'T7', 'T8' ,'T9']
df_T = pd.read_csv(relative_csv_path, skipinitialspace=True, usecols=fields)
X_train, X_test = train_test_split(df_T, test_size = split_ratio)

# Load different G values 
fields = ['G']
df_G = pd.read_csv(relative_csv_path, skipinitialspace=True, usecols=fields)
Y_train, Y_test = train_test_split(df_G, test_size = split_ratio)

# Convert into numpy arrays
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
Y_test = Y_test.to_numpy()
Y_train = Y_train.to_numpy()
Y = df_T.to_numpy()

## Data sample 
$$
X = [T1,T2,T3,T4,T5,T6,T7,T8] \rightarrow G1
$$
## Transform data to this nomenclature:

 $$
     [T_{11}, X_{1}] \rightarrow G_1\\ 
     [T_{12}, X_{2}] \rightarrow G_1\\
     [T_{13}, X_{3}] \rightarrow G_1\\
     [T_{14}, X_{4}] \rightarrow G_1\\
     [T_{15}, X_{5}] \rightarrow G_1\\
     [T_{16}, X_{6}] \rightarrow G_1\\
     [T_{17}, X_{7}] \rightarrow G_1\\
     [T_{18}, X_{8}] \rightarrow G_1\\
     [T_{21}, X_{1}] \rightarrow G_2\\ 
     [T_{22}, X_{2}] \rightarrow G_2\\
     [T_{23}, X_{3}] \rightarrow G_2\\
     [T_{24}, X_{4}] \rightarrow G_2\\
     [T_{25}, X_{5}] \rightarrow G_2\\
     [T_{26}, X_{6}] \rightarrow G_2\\
     [T_{27}, X_{7}] \rightarrow G_2\\
     [T_{29}, X_{8}] \rightarrow G_2
     $$

<!-- 
T2,T3,T4,T5,T6,T7,T8] \rightarrow G1
$$

$
X = [
      [T_1, X_1] [G 
$ -->

In [14]:
def process_data_to_PINN(X_input, Y_output):
    """
    Function to transform the input data to [x1, x2, ... xN] and [T1, T2, ... Tn].

    Parameters:
       X_input (narray): Input array.
       Y_output (narray): Output array.

"""       
    # Split T into a vector
    T_col = X_input.reshape(-1,1)
    N_samples = T_col.shape[0]
    
    # Create a X_col vector
    X_col = np.zeros_like(T_col)

    # Create a G_col vector
    G_col = np.zeros_like(T_col)

    # Fill G and X
    for n in range(T_col.shape[0]):
        X_col[n] = n % 9 / 10 
        G_col[n] = Y_output[n // 9]
    
    return X_col, T_col, G_col

X, T, G = process_data_to_PINN(X_train, Y_train)

# Check outputs
assert(X.shape[0] == T.shape[0] == G.shape[0])
rand_acces = np.random.randint(0,high = T.shape[0], size=1)
assert(X[rand_acces] == X[rand_acces[0] + 9])
assert(G[rand_acces] != G[rand_acces[0] + 9])

In [ ]:

tf.config.experimental.enable_tensor_float_32_execution(False)
#os.environ[‘TF_ENABLE_AUTO_MIXED_PRECISION’] = ‘1’

# Initalization of Network
def hyper_initial(size):
    """
    Initilizes the layer weights according to Xavier procedure. 
    
    Parameters:
       size (integer): Input size.
    
    """
    in_dim = size[0]
    out_dim = size[1]
    std = np.sqrt(2.0/(in_dim + out_dim))
    return tf.Variable(tf.random.truncated_normal(shape=size, stddev = std))

# Neural Network 
def DNN(X, W, b):
    A = X
    L = len(W)
    for i in range(L-1):
        A = tf.tanh(tf.add(tf.matmul(A, W[i]), b[i]))
    Y = tf.add(tf.matmul(A, W[-1]), b[-1])
    return Y

def train_vars(W, b):
    return W + b

def net_u(x, t, w, b):
    u = DNN(tf.concat([x,t],1), w, b)
    return u


#@tf.function(jit_compile=True)
@tf.function
def net_f(x,t,W, b, nu):
    with tf.GradientTape(persistent=True) as tape1:
        tape1.watch([x, t])
        with tf.GradientTape(persistent=True) as tape2:
            tape2.watch([x, t])
            u=net_u(x,t, W, b)
        u_t = tape2.gradient(u, t)
        u_x = tape2.gradient(u, x)
    u_xx = tape1.gradient(u_x, x)  
    del tape1
    f = u_t + u*u_x - nu*u_xx
    return f



#@tf.function(jit_compile=True)
@tf.function
def train_step(W, b, X_u_train_tf, u_train_tf, X_f_train_tf, opt, nu):
    x_u = X_u_train_tf[:,0:1]
    t_u = X_u_train_tf[:,1:2]
    x_f = X_f_train_tf[:,0:1]
    t_f = X_f_train_tf[:,1:2]
    with tf.GradientTape() as tape:
        tape.watch([W,b])
        u_nn = net_u(x_u, t_u, W, b) 
        f_nn = net_f(x_f,t_f, W, b, nu)
        loss =  tf.reduce_mean(tf.square(u_nn - u_train_tf)) + tf.reduce_mean(tf.square(f_nn)) 
    grads = tape.gradient(loss, train_vars(W,b))
    opt.apply_gradients(zip(grads, train_vars(W,b)))
    return loss


    
nu = 0.01/np.pi
noise = 0.0        
N_u = 100
N_f = 10000
Nmax=40000

layers = [2, 20, 20, 20, 20, 20, 20, 20, 20, 1]
L = len(layers)
W = [hyper_initial([layers[l-1], layers[l]]) for l in range(1, L)] 
b = [tf.Variable(tf.zeros([1, layers[l]])) for l in range(1, L)] 

data = scipy.io.loadmat('./Data/burgers_shock.mat')

t = data['t'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = np.real(data['usol']).T
X, T = np.meshgrid(x,t)
X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
u_star = Exact.flatten()[:,None]              
# Doman bounds
lb = X_star.min(0)
ub = X_star.max(0)    
xx1 = np.hstack((X[0:1,:].T, T[0:1,:].T))
uu1 = Exact[0:1,:].T
xx2 = np.hstack((X[:,0:1], T[:,0:1]))
uu2 = Exact[:,0:1]
xx3 = np.hstack((X[:,-1:], T[:,-1:]))
uu3 = Exact[:,-1:]

X_u_train = np.vstack([xx1, xx2, xx3])
X_f_train = lb + (ub-lb)*lhs(2, N_f)
X_f_train = np.vstack((X_f_train, X_u_train))
u_train = np.vstack([uu1, uu2, uu3])

idx = np.random.choice(X_u_train.shape[0], N_u, replace=False)

X_u_train = X_u_train[idx, :]
u_train = u_train[idx,:]

X_u_train_tf = tf.convert_to_tensor(X_u_train, dtype=tf.float32)
u_train_tf =   tf.convert_to_tensor(u_train, dtype=tf.float32)
X_f_train_tf = tf.convert_to_tensor(X_f_train, dtype=tf.float32)

lr = 1e-3
optimizer = tf.optimizers.Adam(learning_rate=lr)

start_time = time.time()
n=0
loss = []
while n <= Nmax:
    loss_= train_step(W, b, X_u_train_tf, u_train_tf, X_f_train_tf, optimizer, nu)
    loss.append(loss_)    
    print(f"Iteration is: {n} and loss is: {loss_}")
    n+=1

elapsed = time.time() - start_time                
print('Training time: %.4f' % (elapsed))
